# 3. Data Preparation – Feature Engineering

Nesta etapa realizamos a consolidação das tabelas relacionais, criando uma visão analítica por cliente.

O objetivo é transformar os dados transacionais em métricas comportamentais (RFM) para posterior modelagem de churn.


In [0]:
df_customers = spark.table("`crisp-dm`.customers")
df_orders = spark.table("`crisp-dm`.orders")
df_order_details = spark.table("`crisp-dm`.order_details")
df_branches = spark.table("`crisp-dm`.branches")
df_categories = spark.table("`crisp-dm`.categories")


## 3.1 Consolidação das Tabelas (Camada Silver)

Nesta etapa realizamos os JOINs entre:

- Orders
- Order_Details
- Customers
- Branches
- Categories

Criando uma base consolidada de vendas.


In [0]:
df_silver = (
    df_orders
    .join(df_order_details, "ORDERID", "inner")
    .join(df_customers, "USERID", "inner")
    .join(df_branches, "BRANCH_ID", "left")
    .join(df_categories, "ITEMID", "left")
)

display(df_silver)


## 3.2 Construção das Métricas RFM

Para análise de churn utilizamos o modelo RFM:

- Recência: dias desde a última compra
- Frequência: número de pedidos
- Monetário: valor total gasto

As métricas serão calculadas por USERID.


In [0]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import DoubleType

# Corrigir vírgula decimal e converter para double
df_silver = df_silver.withColumn(
    "TOTALPRICE",
    regexp_replace(col("TOTALPRICE"), ",", ".").cast(DoubleType())
)

# Criar RFM
from pyspark.sql.functions import max, countDistinct, sum as _sum

df_rfm = (
    df_silver
    .groupBy("USERID")
    .agg(
        max("DATE_").alias("last_purchase"),
        countDistinct("ORDERID").alias("frequency"),
        _sum("TOTALPRICE").alias("monetary")
    )
)

display(df_rfm)


## 3.3 Cálculo da Recência

A recência é calculada como a diferença entre a data atual e a última compra.
Essa métrica permite identificar clientes inativos.


In [0]:
from pyspark.sql.functions import current_date, datediff, to_date, lit

df_rfm = df_rfm.withColumn(
    "last_purchase",
    to_date("last_purchase")
)

reference_date = "2023-08-31"

df_rfm = df_rfm.withColumn(
    "recency",
    datediff(lit(reference_date), df_rfm.last_purchase)
)
display(df_rfm)


## 3.4 Camada Gold

A tabela Gold contém as métricas RFM consolidadas por cliente,
sendo utilizada como base para criação da variável target de churn.


In [0]:
df_rfm.write.mode("overwrite").saveAsTable("`crisp-dm`.gold_rfm")
